# Klasyfikacja binarna obecności osób w pomieszczeniu na podstawie danych sensorowych (Room Occupancy Estimation) #

* Michał Adamiec 217633
* Piotr Lewandowski 217357

### **Streszczenie** ###
Celem projektu jest opracowaneie modelu klasyfikacyjnego umożliwiającego wykrycie osób w pomieszczeniu na podstawie danych wielu nieinwazyjnych czujników środowiskowych, takich jak temperatura, światło, dźwięk, CO2 oraz PIR (pasywny czujnik podczerwieni). Zmienna docelowa została przekształcona do postaci binarnej, co pozwala na rozróżnienie sytuacji, gdy pomieszczenie jest puste lub zajęte.

### **Słowa kluczowe** ###

### Importy

In [ ]:
# function that fetches dataset from ucimlrepo
from ucimlrepo import fetch_ucirepo
import pandas

In [ ]:
# fetch dataset 
room_occupancy_estimation = fetch_ucirepo(id=864) 
  
# data (as pandas dataframes) 
X = room_occupancy_estimation.data.features 
y = room_occupancy_estimation.data.targets 
  
# metadata 
print(room_occupancy_estimation.metadata) 
  
# variable information 
print(room_occupancy_estimation.variables) 
